#Carlos Ramirez Angulo:
### <span style="color:blue">Redes Neurais e Arquiteturas Profundas</span>
### <span style="color:blue">Análise de Dados com Base em Processamento Massivo em Paralelo</span>

### <span style="color:blue">Prova Final</span>

**Material Produzido por:**<br>
>**Profa. Dra. Cristina Dutra de Aguiar**<br>
>**Prof. Dr. Moacir A. Ponti**<br>

**CEMEAI - ICMC/USP São Carlos**


A prova final contém 1 questão, dividida em 3 itens. Por favor, procurem por Questão para encontrar a especificação da questão e por RESOLVER para encontrar a especificação do item a ser solucionado. Também é possível localizar a questão e os itens utilizando o menu de navegação.

O notebook contém a constelação de fatos da BI Solutions que deve ser utilizada para responder à questão e também todas as `bibliotecas`, `bases de dados`, `inicializações`, `instalações`, `importações`, `geração de dataFrames`, `geração de visões temporárias` e `conversão dos tipos de dados` necessárias para a realização da questão.


**INSTRUÇÕES**:<br>
1) Você deve exportar esse notebook com sua solução para as questões da prova em formato .py e fazer upload no Moodle. Atenção: você não deve fazer upload de um arquivo notebook (.ipynb), mas sim um arquivo texto .py contendo os códigos python que utilizou para resolver as questões. O arquivo .py pode ser gerado através da opção:<br>
File --> Download as --> Python (.py)
disponível no Jupyter Notebook.

ou
File --> Download .py
no Google Colab

Caso não esteja utilizando o Jupyter, copie e cole seu código em um arquivo ASCII (Texto) salvando com a extensão .py

2) Você deve salvar esse notebook com sua solução para as questões da prova em formato .pdf e fazer upload no Moodle

3) Os arquivos devem ser nomeados com seu nome e sobrenome, sem espaços. Exemplo: moacirponti.py e moacirponti.pdf

4) É OBRIGATÓRIO conter no cabeçalho (início) do arquivo um comentário / texto com o seu nome completo


**Desejamos uma boa prova!**

#1 Constelação de Fatos da BI Solutions

A aplicação de *data warehousing* da BI Solutions utiliza como base uma contelação de fatos, conforme descrita a seguir.

**Tabelas de dimensão**

- data (dataPK, dataCompleta, dataDia, dataMes, dataBimestre, dataTrimestre, dataSemestre, dataAno)
- funcionario (funcPK, funcMatricula, funcNome, funcSexo, funcDataNascimento, funcDiaNascimento, funcMesNascimento, funcAnoNascimento, funcCidade, funcEstadoNome, funcEstadoSigla, funcRegiaoNome, funcRegiaoSigla, funcPaisNome, funcPaisSigla)
- equipe (equipePK, equipeNome, filialNome, filialCidade, filialEstadoNome, filialEstadoSigla, filialRegiaoNome, filialRegiaoSigla, filialPaisNome, filialPaisSigla)
- cargo (cargoPK, cargoNome, cargoRegimeTrabalho, cargoEscolaridadeMinima, cargoNivel)
- cliente (clientePK, clienteNomeFantasia, clienteSetor, clienteCidade, clienteEstadoNome, clienteEstadoSigla, clienteRegiaoNome, clienteRegiaoSigla, clientePaisNome, clientePaisSigla)

**Tabelas de fatos**
- pagamento (dataPK, funcPK, equipePK, cargoPK, salario, quantidadeLancamentos)
- negociacao (dataPK, equipePK, clientePK, receita, quantidadeNegociacoes)


#2 Configurações


## 2.1 Obtenção dos Dados da BI Solutions

In [ ]:
#instalando o módulo wget
%%capture
!pip install -q wget
!mkdir data

#baixando os dados das tabelas de dimensão e das tabelas de fatos
import wget

url = "https://raw.githubusercontent.com/cristinaaguiar/DataMartBISolutions/main/data.csv"
wget.download(url, "data/data.csv")

url = "https://raw.githubusercontent.com/cristinaaguiar/DataMartBISolutions/main/funcionario.csv"
wget.download(url, "data/funcionario.csv")

url = "https://raw.githubusercontent.com/cristinaaguiar/DataMartBISolutions/main/equipe.csv"
wget.download(url, "data/equipe.csv")

url = "https://raw.githubusercontent.com/cristinaaguiar/DataMartBISolutions/main/cargo.csv"
wget.download(url, "data/cargo.csv")

url = "https://raw.githubusercontent.com/cristinaaguiar/DataMartBISolutions/main/cliente.csv"
wget.download(url, "data/cliente.csv")

url = "https://raw.githubusercontent.com/cristinaaguiar/DataMartBISolutions/main/pagamento.csv"
wget.download(url, "data/pagamento.csv")

url = "https://raw.githubusercontent.com/cristinaaguiar/DataMartBISolutions/main/negociacao.csv"
wget.download(url, "data/negociacao.csv")

## 2.2 Instalações e Inicializações

In [ ]:
#instalando Java Runtime Environment (JRE) versão 8
%%capture
!apt-get remove openjdk*
!apt-get update --fix-missing
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
#baixando Apache Spark versão 3.0.0
%%capture
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop2.7.tgz
!tar xf spark-3.0.0-bin-hadoop2.7.tgz && rm spark-3.0.0-bin-hadoop2.7.tgz

In [ ]:
import os
#configurando a variável de ambiente JAVA_HOME
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
#configurando a variável de ambiente SPARK_HOME
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop2.7"

In [ ]:
%%capture
#instalando o pacote findspark
!pip install -q findspark==1.4.2
#instalando o pacote pyspark
!pip install -q pyspark==3.0.0

## 2.3 Bibliotecas

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("pyspark-notebook").master("local[*]").getOrCreate()

from pyspark.sql.types import IntegerType
from pyspark.sql.types import FloatType
from pyspark.sql.functions import round, desc

import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from numpy.random import seed
from tensorflow.random import set_seed
from tensorflow import keras
from tensorflow.keras import layers

## 2.4 Geração dos DataFrames em Pandas da BI Solutions

Nesta seção são gerados os DataFrames em Pandas. Atenção aos nomes desses DataFrames.

In [ ]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [ ]:
cargoPandas = pd.read_csv('https://raw.githubusercontent.com/cristinaaguiar/DataMartBISolutions/main/cargo.csv')
clientePandas = pd.read_csv('https://raw.githubusercontent.com/cristinaaguiar/DataMartBISolutions/main/cliente.csv')
dataPandas = pd.read_csv('https://raw.githubusercontent.com/cristinaaguiar/DataMartBISolutions/main/data.csv')
equipePandas = pd.read_csv('https://raw.githubusercontent.com/cristinaaguiar/DataMartBISolutions/main/equipe.csv')
funcionarioPandas = pd.read_csv('https://raw.githubusercontent.com/cristinaaguiar/DataMartBISolutions/main/funcionario.csv')
negociacaoPandas = pd.read_csv('https://raw.githubusercontent.com/cristinaaguiar/DataMartBISolutions/main/negociacao.csv')
pagamentoPandas = pd.read_csv('https://raw.githubusercontent.com/cristinaaguiar/DataMartBISolutions/main/pagamento.csv')

## 2.5 Geração dos DataFrames em Spark da BI Solutions

Nesta seção são gerados dos DataFrames em Spark. Atenção aos nomes desses DataFrames.

In [ ]:
#criando os DataFrames em Spark
cargo = spark.read.csv(path="data/cargo.csv", header=True, sep=",")
cliente = spark.read.csv(path="data/cliente.csv", header=True, sep=",")
data = spark.read.csv(path="data/data.csv", header=True, sep=",")
equipe = spark.read.csv(path="data/equipe.csv", header=True, sep=",")
funcionario = spark.read.csv(path="data/funcionario.csv", header=True, sep=",")
negociacao = spark.read.csv(path="data/negociacao.csv", header=True, sep=",")
pagamento = spark.read.csv(path="data/pagamento.csv", header=True, sep=",")

In [ ]:
#convertendo os dados necessários para o tipo de dado inteiro
colunas_cargo = ["cargoPK"]
colunas_cliente = ["clientePK"]
colunas_data = ["dataPK", "dataDia", "dataMes", "dataBimestre", "dataTrimestre", "dataSemestre", "dataAno"]
colunas_equipe = ["equipePK"]
colunas_funcionario = ["funcPK", "funcDiaNascimento", "funcMesNascimento", "funcAnoNascimento"]
colunas_negociacao = ["equipePK", "clientePK", "dataPK", "quantidadeNegociacoes"]
colunas_pagamento = ["funcPK", "equipePK", "dataPK", "cargoPK", "quantidadeLancamentos"]

for coluna in colunas_cargo:
  cargo = cargo.withColumn(coluna, cargo[coluna].cast(IntegerType()))

for coluna in colunas_cliente:
  cliente = cliente.withColumn(coluna, cliente[coluna].cast(IntegerType()))

for coluna in colunas_data:
  data = data.withColumn(coluna, data[coluna].cast(IntegerType()))

for coluna in colunas_equipe:
  equipe = equipe.withColumn(coluna, equipe[coluna].cast(IntegerType()))

for coluna in colunas_funcionario:
  funcionario = funcionario.withColumn(coluna, funcionario[coluna].cast(IntegerType()))

for coluna in colunas_negociacao:
  negociacao = negociacao.withColumn(coluna, negociacao[coluna].cast(IntegerType()))

for coluna in colunas_pagamento:
  pagamento = pagamento.withColumn(coluna, pagamento[coluna].cast(IntegerType()))

In [ ]:
#convertendo os dados necessários para o tipo de dado float
colunas_negociacao = ["receita"]
colunas_pagamento = ["salario"]

for coluna in colunas_negociacao:
  negociacao = negociacao.withColumn(coluna, negociacao[coluna].cast(FloatType()))

for coluna in colunas_pagamento:
  pagamento = pagamento.withColumn(coluna, pagamento[coluna].cast(FloatType()))

In [ ]:
#criando as visões temporárias
cargo.createOrReplaceTempView("cargo")
cliente.createOrReplaceTempView("cliente")
data.createOrReplaceTempView("data")
equipe.createOrReplaceTempView("equipe")
funcionario.createOrReplaceTempView("funcionario")
negociacao.createOrReplaceTempView("negociacao")
pagamento.createOrReplaceTempView("pagamento")

# 3 Questão

A empresa BI Solutions está realizando uma investigação baseada no projeto e treinamento de uma rede neural, usando como base seus dados históricos mantidos na Constelação de Fatos. O modelo resultante deve ser usado para obter uma predição, a qual é voltada à análise da investigação esperada.

**IMPORTANTE**: Leia a questão com muita atenção, desde que vários passos da questão já se encontram implementados. Os locais nos quais os comandos de resposta para os itens da questão devem ser especificados são identificados em comentários.

## 3.1 Investigação dos Dados Históricos

A primeira parte para solucionar a investigação consiste na obtenção de dados históricos de interesse. Isso deve feito por meio da especificação de uma consulta OLAP, segundo as instruções detalhadas a seguir.

### 3.1.1 Instruções para a Especificação da Consulta OLAP


- A **consulta OLAP** pode ser especificada usando qualquer uma das três opções a seguir (escolha somente uma forma)
   - Usando **Pandas** (conceitos apresentados na Aula 05).
   - Usando a **linguagem SQL** (conceitos apresentados na Aula 07).
   - Usando os métodos de **pyspark.sql** (conceitos apresentados na Aula 08).
- Na listagem das respostas:
   - As **colunas** solicitadas devem ser exibidas exatamente na mesma ordem que a definida.
   - As **linhas** retornadas como respostas devem ser exibidas exatamente na mesma ordem que a definida.
   - Os **nomes das colunas** renomeadas devem seguir estritamente os nomes definidos.
-  Quando a consulta OLAP for especificada usando **Pandas**, use:
   - O comando `df.head(25)` para exibir o resultado da consulta. Esse comando deve ser o último comando a ser especificado.
   - O comando `df.round(2)` para arredondar os dados até duas casas decimais.
- Quando a consulta OLAP for especificada usando a **linguagem SQL**, use:
  - O comando `spark.sql(consultaSQL).show(25,truncate=False)` para exibir o resultado da consulta. Esse comando deve ser o último comando a ser especificado.
  - A função `ROUND(funçãoDeAgregação,2)` para arredondar o dado até duas casas decimais.
- Quando a consulta OLAP for especificada usando os demais **métodos de pyspark.sql**, use:
  - O comando `nomeDoDataFrame.show(25,truncate=False)` para exibir o resultado da consulta. Esse comando deve ser o último comando a ser especificado.
  - O método `round(funçãoDeAgregação,2)` para arredondar o dado até duas casas decimais.

### 3.1.2 Consulta OLAP (RESOLVER)

Liste a soma dos salários por mês, ano e nome do cargo, considerando os funcionários que moram na região SUDESTE e os anos de 2019 e 2020. Arredonde a soma dos salários para até duas casas decimais. Devem ser exibidas as colunas na ordem e com os nomes especificados a seguir: "MES", "ANO", "CARGO" e "TOTALSALARIO". Ordene as linhas exibidas primeiro por mês, depois por ano, depois por nome do cargo, todos em ordem ascendente. Liste as primeiras 25 linhas da resposta, sem truncamento das strings.


In [ ]:
# Escreva aqui a sua resposta para a consulta OLAP

query = """
SELECT
dataMes AS MES,
dataAno AS ANO,
 cargoNome AS CARGO,
  ROUND(SUM(salario), 2) AS TOTALSALARIO
FROM pagamento JOIN data ON data.dataPK = pagamento.dataPK
               JOIN cargo ON cargo.cargoPK = pagamento.cargoPK
               JOIN funcionario ON funcionario.funcPK = pagamento.funcPK
WHERE funcRegiaoNome = 'SUDESTE'
  AND dataAno >= 2019
  AND dataAno <= 2020
GROUP BY dataMes, dataAno, cargoNome
ORDER BY dataMes ASC, dataAno ASC, cargoNome ASC
"""

# spark.sql(query).show()
spark.sql(query).show(25, truncate=False)

+---+----+-----------------------------------------------------------+------------+
|MES|ANO |CARGO                                                      |TOTALSALARIO|
+---+----+-----------------------------------------------------------+------------+
|1  |2019|ADMINISTRADOR DE SISTEMAS COMPUTACIONAIS                   |14715.22    |
|1  |2019|ADMINISTRADOR DE SISTEMAS OPERACIONAIS DE REDE             |22613.39    |
|1  |2019|ADMINISTRADOR EM SEGURANCA DA INFORMACAO                   |119829.04   |
|1  |2019|ANALISTA DE REDE                                           |100923.66   |
|1  |2019|ANALISTA DE SISTEMAS (INFORMATICA)                         |66014.69    |
|1  |2019|ANALISTA DE SISTEMAS PARA INTERNET                         |27221.96    |
|1  |2019|ANALISTA DE SUPORTE DE BANCO DE DADOS                      |40552.49    |
|1  |2019|ANALISTA DE SUPORTE TECNICO                                |161771.0    |
|1  |2019|CIENTISTA DE DADOS (DATA SCIENTIST)                        |51170.

In [ ]:
# Se a sua consulta foi especificada usando a linguagem SQL
# Transforme o resultado da consulta em um dataFrame em Pandas
# descomentando o seguinte comando:
df = spark.sql(query).toPandas()

In [ ]:
# Se a sua consulta foi especificada usando os comandos de pyskpark
# Transforme o resultado da consulta em um dataFrame em Pandas
# descomentado o seguinte comando:
#df = dataFramepyspark.toPandas()

In [ ]:
# Exibindo algumas linhas do dataFrame gerado
# Note que o dataFrame gerado é chamado de df
df

,MES,ANO,CARGO,TOTALSALARIO
0,1,2019,ADMINISTRADOR DE SISTEMAS COMPUTACIONAIS,14715.22
1,1,2019,ADMINISTRADOR DE SISTEMAS OPERACIONAIS DE REDE,22613.39
2,1,2019,ADMINISTRADOR EM SEGURANCA DA INFORMACAO,119829.04
3,1,2019,ANALISTA DE REDE,100923.66
4,1,2019,ANALISTA DE SISTEMAS (INFORMATICA),66014.69
...,...,...,...,...
427,12,2020,OPERADOR DE CENTRO DE PROCESSAMENTO DE DADOS,58871.18
428,12,2020,PROGRAMADOR DE APLICATIVOS EDUCACIONAIS E DE E...,67313.13
429,12,2020,PROGRAMADOR DE SISTEMAS DE COMPUTADOR,73333.11
430,12,2020,PROGRAMADOR DE SISTEMAS DE INFORMACAO,142442.45


## 3.2 Treinamento da Rede Neural

A segunda parte para solucionar a investigação consiste no treinamento da rede neural usando como base os dados históricos obtidos no item 3.1.2, segundo as instruções detalhadas a seguir.


### 3.2.1 Preparação dos Dados

A preparação dos dados já encontra-se pronta, sendo necessário apenas executar as células abaixo.

In [ ]:
df.columns, df.shape

(Index(['MES', 'ANO', 'CARGO', 'TOTALSALARIO'], dtype='object'), (432, 4))

In [ ]:
df.index

RangeIndex(start=0, stop=432, step=1)

In [ ]:
# ordena dados
df = df.sort_values(by=['ANO', 'MES'])

# obtem dummy-variables / one-hot-encoding
ano = pd.get_dummies(df['ANO'])
mes = pd.Series((df['MES'].values > 6).astype(int), name='MES')
cargo = pd.get_dummies(df['CARGO'])

# cria dataframe para treinamento
input_data = pd.concat([ano, mes, cargo, df['TOTALSALARIO']], axis=1)

In [ ]:
input_data

,2019,2020,MES,ADMINISTRADOR DE SISTEMAS COMPUTACIONAIS,ADMINISTRADOR DE SISTEMAS OPERACIONAIS DE REDE,ADMINISTRADOR EM SEGURANCA DA INFORMACAO,ANALISTA DE REDE,ANALISTA DE SISTEMAS (INFORMATICA),ANALISTA DE SISTEMAS PARA INTERNET,ANALISTA DE SUPORTE DE BANCO DE DADOS,...,DBA,DESENVOLVEDOR MOBILE,DESIGNER DE GAMES,ENGENHEIRO DE DADOS,OPERADOR DE CENTRO DE PROCESSAMENTO DE DADOS,PROGRAMADOR DE APLICATIVOS EDUCACIONAIS E DE ENTRETENIMENTO,PROGRAMADOR DE SISTEMAS DE COMPUTADOR,PROGRAMADOR DE SISTEMAS DE INFORMACAO,PROGRAMADOR FRONT-END,TOTALSALARIO
0,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,14715.22
1,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,22613.39
2,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,119829.04
3,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,100923.66
4,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,66014.69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,58871.18
428,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,67313.13
429,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,73333.11
430,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,142442.45


In [ ]:
# separa dados de treinamento e 5 para teste
n_test = 5

x_train = input_data.iloc[:-n_test,:-1]
x_test = input_data.iloc[-n_test:,:-1]

y_train = input_data.iloc[:-n_test,-1]
y_test = input_data.iloc[-n_test:,-1]

print("Tamanho conjunto de treinamento: ", x_train.shape, y_train.shape)
print("Tamanho conjunto de testes: ", x_test.shape, y_test.shape)

Tamanho conjunto de treinamento:  (427, 21) (427,)
Tamanho conjunto de testes:  (5, 21) (5,)


In [ ]:
x_test

,2019,2020,MES,ADMINISTRADOR DE SISTEMAS COMPUTACIONAIS,ADMINISTRADOR DE SISTEMAS OPERACIONAIS DE REDE,ADMINISTRADOR EM SEGURANCA DA INFORMACAO,ANALISTA DE REDE,ANALISTA DE SISTEMAS (INFORMATICA),ANALISTA DE SISTEMAS PARA INTERNET,ANALISTA DE SUPORTE DE BANCO DE DADOS,...,CIENTISTA DE DADOS (DATA SCIENTIST),DBA,DESENVOLVEDOR MOBILE,DESIGNER DE GAMES,ENGENHEIRO DE DADOS,OPERADOR DE CENTRO DE PROCESSAMENTO DE DADOS,PROGRAMADOR DE APLICATIVOS EDUCACIONAIS E DE ENTRETENIMENTO,PROGRAMADOR DE SISTEMAS DE COMPUTADOR,PROGRAMADOR DE SISTEMAS DE INFORMACAO,PROGRAMADOR FRONT-END
427,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
428,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
429,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
430,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
431,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
# valor maximo de treinamento para normalizacao de salario para 0-1
y_max = np.max(y_train)
print('Max train', y_max)

Max train 161771.0


In [ ]:
# dados para regressao normalizados
y_train_n = np.array(y_train)/y_max
y_test_n = np.array(y_test)/y_max

### 3.2.2 Projeto e Treinamento de Rede Neural (RESOLVER)

Utilize a função a seguir como base, projetando uma rede para realizar regressão do salário a partir de ano, sexo e cidade. Utilizar a seguinte arquitetura:
* entrada com ativação relu
* 3 (três) camadas densas com 16 neurônios, ativação relu
* camada dropout com taxa 0.275
* uma camada densa de saída (regressão)

Instancie, compile e treine a rede neural utilizando:
* otimizador AdamW
* função de custo de erro médio quadrático,
* learning rate inicial de 0.002 e decaimento dado pela função `scheduler` provida,
* 35 épocas,
* batchsize de tamanho 10.

OBS: para obter a resposta, após implementar tudo, reinicie o Kernel do notebook e execute tudo novamente em ordem

In [ ]:
# Funcao base a ser usada para instancia a rede neural
# a funcao NAO precisa ser modificada, apenas é necessário
# chamá-la com os parametros corretos conforme instruções acima
def deep_net(neurons, input_dim, n_layers=1, batch_norm=False, dropout_rate=0.0):

    input_data = keras.layers.Input(shape=(input_dim,))

    x = keras.layers.Activation('relu')(input_data)
    if batch_norm:
        x = keras.layers.BatchNormalization(name='bn_input')(x)
        x = keras.layers.Activation('relu')(x)
    x = keras.layers.Dense(neurons, activation='relu')(x)
    for i in range(n_layers-1):
      x = keras.layers.Dense(neurons, activation='relu')(x)
    x = keras.layers.Dropout(dropout_rate)(x)
    output = keras.layers.Dense(1, activation='relu')(x)
    dnn = keras.models.Model(input_data, output)

    return dnn

In [ ]:
def scheduler(epoch, lr):
    if epoch <= 3: return lr
    return lr * tf.math.exp(-0.05)

callbacklr = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [ ]:
seed(1)
set_seed(2)

In [ ]:
# Escreva aqui a sua resposta para instanciar, compilar e treinar a rede neural

model = deep_net(neurons=16, input_dim=x_train.shape[1], n_layers=3, dropout_rate=0.275)
optimizer = tf.keras.optimizers.AdamW(learning_rate=0.002)
model.compile(optimizer=optimizer, loss='mean_squared_error')
model.fit(x_train, y_train_n, epochs=35, batch_size=10, callbacks=[callbacklr])

Epoch 1/35
43/43 [==============================] - 1s 2ms/step - loss: 0.0997 - lr: 0.0020
Epoch 2/35
43/43 [==============================] - 0s 2ms/step - loss: 0.0552 - lr: 0.0020
Epoch 3/35
43/43 [==============================] - 0s 2ms/step - loss: 0.0332 - lr: 0.0020
Epoch 4/35
43/43 [==============================] - 0s 2ms/step - loss: 0.0186 - lr: 0.0020
Epoch 5/35
43/43 [==============================] - 0s 2ms/step - loss: 0.0172 - lr: 0.0019
Epoch 6/35
43/43 [==============================] - 0s 2ms/step - loss: 0.0113 - lr: 0.0018
Epoch 7/35
43/43 [==============================] - 0s 2ms/step - loss: 0.0128 - lr: 0.0017
Epoch 8/35
43/43 [==============================] - 0s 2ms/step - loss: 0.0113 - lr: 0.0016
Epoch 9/35
43/43 [==============================] - 0s 2ms/step - loss: 0.0132 - lr: 0.0016
Epoch 10/35
43/43 [==============================] - 0s 2ms/step - loss: 0.0098 - lr: 0.0015
Epoch 11/35
43/43 [==============================] - 0s 2ms/step - loss: 0.0101

### 3.2.3 Predição de salário (RESOLVER)

Utilizando o modelo treinado, realize a predição do salário para as linhas do conjunto de teste, i.e. em `x_test_n`, e calcule a diferença absoluta entre os valores verdadeiros e preditos.

Não se esqueça de, após obter a predição, reescalar novamente para o intervalo original dos dados, revertendo a normalização feita.

A resposta na questão é equivalente ao valor obtido em milhares de reais.

In [ ]:
# Escreva aqui a sua resposta para a predição de salário

y_pred = model.predict(x_test)
y_pred *= y_max

erro_absoluto = np.abs(y_test.to_numpy() - y_pred.reshape((1, -1))[0])
erro_absoluto.mean()

1/1 [==============================] - 0s 102ms/step


4150.962531250004

In [ ]:
# Resultado do questionário

y_test.to_numpy()[erro_absoluto.argmax()]

# Número de cargos distintos = 18, Salário relativo à maior diferença com relação ao predito=142mil

142442.45